In [3]:
import cv2
import json
import numpy as np

def between(a, x,  b):
    return a < x and x <= b

def get_direction_label(degree):
    
    if between(348.75, degree, 360) or between(0, degree, 11.25) or degree == 0:
        return "N"
    elif between(11.25, degree, 33.75):
        return "NNE"
    elif between(33.75, degree, 56.25):
        return "NE"
    elif between(56.25, degree, 78.75):
        return "ENE"
    elif between(78.75, degree, 101.25):
        return "E"
    elif between(101.25, degree, 123.75):
        return "ESE"
    elif between(123.75, degree, 146.25):
        return "SE"
    elif between(146.25, degree, 168.75):
        return "SSE"
    elif between(168.75, degree, 191.25):
        return "S"
    elif between(191.25, degree, 213.75):
        return "SSW"
    elif between(213.75, degree, 236.25):
        return "SW"
    elif between(236.25, degree, 258.75):
        return "WSW"
    elif between(258.75, degree, 281.25):
        return "W"
    elif between(281.25, degree, 303.75):
        return "WNW"
    elif between(303.75, degree, 326.25):
        return "NW"
    elif between(326.25, degree, 348.75):
        return "NNW"
    else:
        print("error.", degree) # There is an instance with 399, should be mistyped
        return 'NE'

def create_color_map():
    # Assign a unique color to each class
    color_map = {
        'BG': (0, 0, 0),   # Background
        'flat': (128, 0, 0),
        'dome': (0, 128, 0),
        'N': (0, 0, 128),
        'NNE': (128, 128, 0),
        'NE': (0, 128, 128),
        'ENE': (128, 0, 128),
        'E': (255, 0, 0),
        'ESE': (0, 255, 0),
        'SE': (0, 0, 255),
        'SSE': (255, 255, 0),
        'S': (0, 255, 255),
        'SSW': (255, 0, 255),
        'SW': (128, 128, 128),
        'WSW': (192, 192, 192),
        'W': (64, 0, 0),
        'WNW': (192, 192, 0),
        'NW': (0, 64, 0),
        'NNW': (0, 192, 0),
        'tree': (0, 0, 255)
    }
    return color_map

def create_mask(image_shape, polygons, color_map):
    mask = np.zeros(image_shape + (3,), dtype=np.uint8)
    for region_id, region_data in polygons.items():
        shape_attributes = region_data['shape_attributes']
        points_x = shape_attributes['all_points_x']
        points_y = shape_attributes['all_points_y']
        class_label = region_data['region_attributes']['building']
        if class_label not in ["tree", "flat"]:
            class_label = get_direction_label(int(class_label))
        color = color_map.get(class_label, (0, 0, 0))

        pts = np.array(list(zip(points_x, points_y)), dtype=np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.fillPoly(mask, [pts], color)

    return mask

# Load VIA annotations from JSON file
folders = ['train', 'test', 'val']
for folder in folders:

    with open(f'deeproof-release/data/deeproof/{folder}/via_region_data.json') as json_file:
        data = json.load(json_file)
    
    # Create a color map
    color_map = create_color_map()
    
    # Iterate through each image in the dataset
    for image_filename, image_data in data.items():
        image_shape = (512,512)
        polygons = image_data['regions']
    
        # Create mask for the current image
        mask = create_mask(image_shape, polygons, color_map)
    
        # Save the mask as an image
        mask_filename = f"mask_{image_filename.replace('.jpg', '.png')}"
        mask_filename = mask_filename.split('.png')[0] + '.png'
        cv2.imwrite(f'deeproof-release/data/deeproof/{folder}/{mask_filename}', mask)
        
        
# Explanation for preprocess : The dataset format was VIA (VGG Image Annotator), so we changed the format for segmentation maps, In the dataset there are 20 images with not annotated , #TODO so we removed these.


error. 399
